In [3]:
import requests
import pandas as pd
import json
import re
import polars as pl
pl.Config.set_tbl_rows(1000)
from persiantools import characters, digits
import sqlite3

In [4]:
BASE_URL = 'https://codal.ir'

In [5]:
symbol = "فخوز"
report_date = 14031230
parse_date = f"{str(report_date)[:4]}%2F{str(report_date)[4:6]}%2F{str(report_date)[6:]}"

letter_type = 58 #6 -> statements #58 -> monthly reports
category = 3 #1 -> statements #3 -> monthly

url = f"https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category={category}&Childs=false&CompanyState=0&CompanyType=-1&Consolidatable=true&IsNotAudited=false&Length=-1&LetterType={letter_type}&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber=1&Publisher=false&ReportingType=-1&Symbol={symbol}&TracingNo=-1&YearEndToDate={parse_date}&search=true"

payload = {}
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:144.0) Gecko/20100101 Firefox/144.0',
  'Accept': 'application/json, text/plain, */*',
  'Accept-Language': 'en-US,en;q=0.5',
  'Accept-Encoding': 'gzip, deflate, br, zstd',
  'Origin': 'https://www.codal.ir',
  'Connection': 'keep-alive',
  'Referer': 'https://www.codal.ir/',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-site',
  'Cookie': 'TS018fb0f7=01f9930bd2042dcf9847aa2bc829c3c06ffc32f99581aeb3be233ae7927f334551012da7bebd35160e4cfe5b1483035393c7c97f49; Unknown=1076170924.20480.0000'
}

response = requests.request("GET", url, headers=headers, data=payload)

results=response.json()

In [6]:
results

{'Total': 15,
 'Page': 1,
 'Letters': [{'SuperVision': {'UnderSupervision': 1,
    'AdditionalInfo': '',
    'Reasons': ['عدم ارائه صورتهای مالی تلفیقی میاندوره ای ۶ ماهه حسابرسی شده',
     'عدم ارائه گزارش تفسیری مدیریت میاندوره ای ۶ ماهه حسابرسی شده']},
   'TracingNo': 1351666,
   'Symbol': 'فخوز',
   'CompanyName': 'فولاد خوزستان',
   'UnderSupervision': 0,
   'Title': 'گزارش فعالیت ماهانه دوره ۱ ماهه منتهی به  ۱۴۰۳/۱۲/۳۰(اصلاحیه)',
   'LetterCode': 'ن-۳۰',
   'SentDateTime': '۱۴۰۴/۰۳/۰۶ ۱۵:۴۲:۰۸',
   'PublishDateTime': '۱۴۰۴/۰۳/۰۶ ۱۵:۴۲:۰۸',
   'HasHtml': True,
   'IsEstimate': False,
   'Url': '/Reports/Decision.aspx?LetterSerial=Mc0qOsUl7HwIeu3mKWzVOOObOOOQ%3d%3d&rt=0&let=58&ct=0&ft=-1',
   'HasExcel': True,
   'HasPdf': True,
   'HasXbrl': False,
   'HasAttachment': True,
   'AttachmentUrl': '/Reports/Attachment.aspx?LetterSerial=Mc0qOsUl7HwIeu3mKWzVOOObOOOQ%3d%3d',
   'PdfUrl': 'DownloadFile.aspx?hs=Mc0qOsUl7HwIeu3mKWzVOOObOOOQ%3d%3d&ft=1005&let=58',
   'ExcelUrl': 'https://exc

In [19]:
#period = 0
# for rr in results['Letters']:
#     for res in str(rr['Title']).split(): 
#         try:

#             if res == 'سال':
#                 period = 12

#             elif isinstance(int(res), int):
#                 period = digits.fa_to_en(res)
#             print(period)
#         except:
#             continue

In [4]:
def getPeriod(results: list) -> int:
    for res in results: 
        try:
            if res == 'سال':
                return 12
            elif isinstance(int(res), int):
                return digits.fa_to_en(res)           
        except:
            continue 

In [5]:
from persiantools import digits, characters
reports={}
for result in results['Letters']:
    res_list = str(result['Title']).split()
    for res in res_list:
        # if res == 'سال':
        #     period = 12
        # elif len(res) == 1 and res!='و':
        #     period = digits.fa_to_en(res)
        if '/' in res:
            r = int(digits.fa_to_en(res).replace('/',''))
            if r not in reports.keys():
                period = getPeriod(res_list)
                p = int(digits.fa_to_en(result['PublishDateTime'].split()[0]).replace('/',''))
                reports[r] = {'period': period, 'publish': p, 'title': result['Title'], 'url': BASE_URL+result['Url']}
reports


{14031230: {'period': 12,
  'publish': 14040506,
  'title': 'صورت\u200cهای مالی تلفیقی سال مالی منتهی به ۱۴۰۳/۱۲/۳۰ (حسابرسی شده)',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=1z105r5OIg1Yv0h84qhcAw%3d%3d&rt=0&let=6&ct=0&ft=-1'},
 14030930: {'period': '9',
  'publish': 14031030,
  'title': 'اطلاعات و صورت\u200cهای مالی میاندوره\u200cای  دوره ۹ ماهه منتهی به  ۱۴۰۳/۰۹/۳۰ (حسابرسی نشده)',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=YQQQaQQQr9ls8uXPQkIRgPeupRng%3d%3d&rt=0&let=6&ct=0&ft=-1'},
 14030631: {'period': '6',
  'publish': 14031017,
  'title': 'اطلاعات و صورت\u200cهای مالی میاندوره\u200cای تلفیقی دوره ۶ ماهه منتهی به  ۱۴۰۳/۰۶/۳۱ (حسابرسی شده)',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=QQQaQQQxQbIEtDUrTV45Sat70Zzw%3d%3d&rt=0&let=6&ct=0&ft=-1'},
 14030331: {'period': '3',
  'publish': 14030430,
  'title': 'اطلاعات و صورت\u200cهای مالی میاندوره\u200cای  دوره ۳ ماهه منتهی به  ۱۴۰۳/۰۳/۳۱ (حسابرسی نشده)',
  'url': 'https://codal.ir

In [22]:
# sort from newest to the oldest
#for date, report in reports.items():
    #print(report['url']+Sheets.sheets.Operational.value)
#date = 14030930
#url = reports[date]['url']+Sheets.sheets.Operational.value

In [6]:
# sort from oldest to the newest
for date in sorted(reports.keys(), reverse=False):
    report = reports[date]
    print(report['url'] + Sheets.sheets.Operational.value)

https://codal.ir/Reports/Decision.aspx?LetterSerial=6AvlPFy8ttXf0xGo3Ma5lg%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20
https://codal.ir/Reports/Decision.aspx?LetterSerial=QQQaQQQxQbIEtDUrTV45Sat70Zzw%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20
https://codal.ir/Reports/Decision.aspx?LetterSerial=YQQQaQQQr9ls8uXPQkIRgPeupRng%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20
https://codal.ir/Reports/Decision.aspx?LetterSerial=1z105r5OIg1Yv0h84qhcAw%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20


In [ ]:
payload = {}
headers = {
 
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:143.0) Gecko/20100101 Firefox/143.0',
  'Accept': 'application/json, text/plain, */*',
  'Accept-Language': 'en-US,en;q=0.5',
  'Accept-Encoding': 'gzip, deflate, br, zstd',
  'Origin': 'https://www.codal.ir',
  'Connection': 'keep-alive',
  'Referer': 'https://www.codal.ir/',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-site',
  'Cookie': 'TS018fb0f7=01f9930bd2e2675d04882f623c888052df60031e7775d8e3c459dc4cb96bf8e870e0a9bf0e0eedff68bcce23fd9701c7a7fc0855c4; Unknown=1076170924.20480.0000'
}

response = requests.request("GET", url)


In [7]:
statement = response.text
pattern = r"var datasource = (.*?});"
match = re.search(pattern, statement)
if match:
    text = match.group(1)
records = []
records.append(
    (statement, text))
for letter, data in records:
    continue
items = json.loads(data)['sheets']
display(items)

NameError: name 'text' is not defined

In [ ]:
# from persiantools import digits
# reports = {}
# for result in results['Letters']:
#     for t in result['Title'].split():
#         if '/' in t:
#             t = digits.fa_to_en(t).replace('/', '')
#             if t not in reports.keys():
#                 reports[t] = {'title': result['Title'], 'url': BASE_URL+result['Url']}
# reports

In [ ]:
items[0]['tables'][5]['cells'] #bahaye tamam shode

In [ ]:
cells = items[0]['tables'][1]['cells']
print(cells[-1]['columnSequence'])
print(cells[-1]['rowSequence'])
print(cells[-1]['value'])
print(cells[-1]['periodEndToDate'])

26
21
1281490000
1403/12/30


In [ ]:
def has_empty_string(df: pl.DataFrame) -> bool:
    return(
        df.select(
            pl.any_horizontal([pl.col(c) == "" for c in df.columns])
        )
        .select(pl.col("*").any())
        .item()
    )

In [ ]:
cells = items[0]['tables'][1]['cells']
cells_ = [(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in cells]
dates = sorted(list(set([i[-1] for i in cells_ if i[-1] != ''])))


all_data ={'report_last_year': pl.DataFrame(),
           'report_this_year': pl.DataFrame(),
           'est_remain': pl.DataFrame(),
           'est_next_year': pl.DataFrame()}

for date_ in dates:
    filtered_cells = [(i[0], i[1], i[2]) for i in cells_ if i[-1] == '' or i[-1] == (date_)]
    df = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
    df = df.pivot(values="value", on="col", index="row").sort("row")
    df = df.with_columns(pl.col('1').map_elements(characters.ar_to_fa, return_dtype=pl.String))
    if(df.width < 8):
        continue
    df_internal = df.clone()
    idx_internal = df_internal.filter(df_internal['1'] == "فروش داخلی:")['row'][0]
    idx_total_internal = df_internal.filter(df_internal['1'] == "جمع فروش داخلی")['row'][0]
    df_internal = df_internal.filter(df_internal['row'] > idx_internal, df_internal['row'] < idx_total_internal-1)[['1','2']]
    df_internal=df_internal.with_columns(pl.lit("داخلی").alias("3"))
    df_export=df.clone()
    idx_export = df_export.filter(df_export['1'] == "فروش صادراتی:")['row'][0]
    idx_total_export = df_export.filter(df_export['1'] == "جمع فروش صادراتی")['row'][0]
    df_export = df_export.filter(df_export['row']>idx_export ,df_export['row']<idx_total_export -1)[['1','2']]
    df_export = df_export.with_columns(pl.lit("صادراتی").alias('3'))
    df_products=pl.concat([df_internal,df_export])
    
    for i in df[0].rows(named=True):
        for k, v in i.items():
            if '/' in str(v):
                test_str = v
                col_num_production = k
                dummy_data = df[col_num_production:str(int(col_num_production)+2)]
                dummy_data = dummy_data.insert_column(0, df['row'])
                           
                sales_internal = dummy_data.filter(dummy_data['row'] > idx_internal, dummy_data['row'] < idx_total_internal-1)
                sales_export = dummy_data.filter(dummy_data['row']>idx_export ,dummy_data['row']<idx_total_export -1)
                sales_products = pl.concat([sales_internal, sales_export])
                sales_products=sales_products.drop('row')

                #Rename columns to range(0,1,...)
                sales_products = sales_products.rename(dict(zip(sales_products.columns, map(str, range(len(sales_products.columns))))))
                #If "" is in the dataframe
                if has_empty_string(sales_products):
                    continue
                #Convert string to Int from Col 1 to end
                sales_products = sales_products.with_columns([pl.col(sales_products.columns).cast(pl.Int32)])
                # concat label and values
                df_products = df_products.rename({c: "_" + c for c in df_products.columns})
                
                data = pl.concat([df_products, sales_products], how="horizontal")
                data.columns = [str(i) for i in range(data.width)]
                data = data.insert_column(0, pl.lit(symbol).alias("symbol"))
                data = data.insert_column(1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("date"))
                data = data.insert_column(2, pl.lit(int(reports[date]['period'])).alias("period"))
                data = data.insert_column(3, pl.lit(reports[date]['publish']).alias("publish"))
                data = data.rename({
                    "0": "product",
                    "1": "unit",
                    "2":"type",
                    "3":"production",
                    "4":"Sales",
                    "5":"price"
                })
                
                test_dict = {
                     "فخوز": 1020,
                     "بلوم ، بیلت ": 1,
                     "اسلب":2,
                     "آهن اسفنجی":3,
                     "گندله ":4,
                     "داخلی":1,
                     "صادراتی":2,
                     "تن":1
                 }  
                data = data.with_columns(
                    pl.col(['symbol','product','unit','type']).replace(test_dict).cast(pl.Int32)
                )
                # Create KEY -> date+symbol_code+product+type -> int64
                data = data.with_columns((pl.col("date").cast(pl.Utf8)+ pl.col("symbol").cast(pl.Utf8)+pl.col("product").cast(pl.Utf8)+pl.col("symbol").cast(pl.Utf8)+pl.col("type").cast(pl.Utf8)).alias("key").cast(pl.Int64))                  
                data = data.select(["key", *[c for c in data.columns if c != "key"]])  
                cells = items[0]['tables'][1]['cells']
cells_ = [(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in cells]
dates = sorted(list(set([i[-1] for i in cells_ if i[-1] != ''])))
num_tables=len(dates)

all_data ={'report_last_year': pl.DataFrame(),
           'report_this_year': pl.DataFrame(),
           'est_remain': pl.DataFrame(),
           'est_next_year': pl.DataFrame()}

for date_ in dates:
    filtered_cells = [(i[0], i[1], i[2]) for i in cells_ if i[-1] == '' or i[-1] == (date_)]
    df = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
    df = df.pivot(values="value", on="col", index="row").sort("row")
    df = df.with_columns(pl.col('1').map_elements(characters.ar_to_fa, return_dtype=pl.String))
    if(df.width < 8):
        continue
    df_internal = df.clone()
    idx_internal = df_internal.filter(df_internal['1'] == "فروش داخلی:")['row'][0]
    idx_total_internal = df_internal.filter(df_internal['1'] == "جمع فروش داخلی")['row'][0]
    df_internal = df_internal.filter(df_internal['row'] > idx_internal, df_internal['row'] < idx_total_internal-1)[['1','2']]
    df_internal=df_internal.with_columns(pl.lit("داخلی").alias("3"))
    df_export=df.clone()
    idx_export = df_export.filter(df_export['1'] == "فروش صادراتی:")['row'][0]
    idx_total_export = df_export.filter(df_export['1'] == "جمع فروش صادراتی")['row'][0]
    df_export = df_export.filter(df_export['row']>idx_export ,df_export['row']<idx_total_export -1)[['1','2']]
    df_export = df_export.with_columns(pl.lit("صادراتی").alias('3'))
    df_products=pl.concat([df_internal,df_export])
    
    for i in df[0].rows(named=True):
        for k, v in i.items():
            if '/' in str(v):
                test_str = v
                col_num_production = k
                dummy_data = df[col_num_production:str(int(col_num_production)+2)]
                dummy_data = dummy_data.insert_column(0, df['row'])
                           
                sales_internal = dummy_data.filter(dummy_data['row'] > idx_internal, dummy_data['row'] < idx_total_internal-1)
                sales_export = dummy_data.filter(dummy_data['row']>idx_export ,dummy_data['row']<idx_total_export -1)
                sales_products = pl.concat([sales_internal, sales_export])
                sales_products=sales_products.drop('row')
                

                #Rename columns to range(0,1,...)
                sales_products = sales_products.rename(dict(zip(sales_products.columns, map(str, range(len(sales_products.columns))))))
                #If "" is in the dataframe
                if has_empty_string(sales_products):
                    continue
                #Convert string to Int from Col 1 to end
                sales_products = sales_products.with_columns([pl.col(sales_products.columns).cast(pl.Int32)])
                # concat label and values
                df_products = df_products.rename({c: "_" + c for c in df_products.columns})
                
                data = pl.concat([df_products, sales_products], how="horizontal")
                data.columns = [str(i) for i in range(data.width)]
                data = data.insert_column(0, pl.lit(symbol).alias("symbol"))
                data = data.insert_column(1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("date"))
                data = data.insert_column(2, pl.lit(int(reports[date]['period'])).alias("period"))
                data = data.insert_column(3, pl.lit(reports[date]['publish']).alias("publish"))
                data = data.rename({
                    "0": "product",
                    "1": "unit",
                    "2":"type",
                    "3":"production",
                    "4":"Sales",
                    "5":"price"
                })
                
                test_dict = {
                     "فخوز": 1020,
                     "بلوم ، بیلت ": 1,
                     "اسلب":2,
                     "آهن اسفنجی":3,
                     "گندله ":4,
                     "داخلی":1,
                     "صادراتی":2,
                     "تن":1
                 }  
                data = data.with_columns(
                    pl.col(['symbol','product','unit','type']).replace(test_dict).cast(pl.Int32)
                )
                # Create KEY -> date+symbol_code+product+type -> int64
                data = data.with_columns((pl.col("date").cast(pl.Utf8)+ pl.col("symbol").cast(pl.Utf8)+pl.col("product").cast(pl.Utf8)+pl.col("symbol").cast(pl.Utf8)+pl.col("type").cast(pl.Utf8)).alias("key").cast(pl.Int64))                  
                data = data.select(["key", *[c for c in data.columns if c != "key"]])  
                if int(str(date_).replace('/','')) < date:
                    all_data['report_last_year'] = data
                elif int(str(date_).replace('/','')) == date:
                    all_data['report_this_year'] = data
                else:
                    if(str(v).split()[-1][:4] == str(date)[:4]):
                        all_data['est_remain'] = data
                    else:
                        all_data["est_next_year"] = data
display(all_data)

In [ ]:
all_data['report_last_year']

In [ ]:
test_str=""

In [2]:
if '/' in test_str.split()[-1]:
    print(test_str)

NameError: name 'test_str' is not defined

In [ ]:
 # Add publish Column
# # Report 3,6,9 test as well
 # ***
# Estimates should be in seperate tables
# Concat Dfs into one but Reports(3,6,9,12) and Estimates(3,6,9,12) -> in next year estimate should be updated
# 
 # $$$
 # # Standard Products


In [ ]:
cells = items[0]['tables'][5]['cells']
cells_ = [(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in cells]
dates = sorted(list(set([i[-1] for i in cells_ if i[-1] != ''])))
for date in dates:
    filtered_cells = [(i[0], i[1], i[2]) for i in cells_ if i[-1] == '' or i[-1] == (date)]
    df1 = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
    df1 = df1.pivot(values="value", on="col", index="row").sort("row")
    df1 = df1.with_columns(pl.col('1').map_elements(characters.ar_to_fa, return_dtype=pl.String))
df1
    

In [ ]:
import sqlite3
conn =sqlite3.connect('codal.sqlite')
cur=conn.cursor()

cur.execute('''drop table if exists report_year''')
cur.execute('''create table repor_year(key INTEGER,
            symbol INTEGER, date INTEGER, period INTEGER,
            publish INTEGER, product INTEGER,unit INTEGER,
            type INTEGER, production INTEGER, Sales INTEGER)''')
cur.execute('''drop table if exists EST_year''')
cur.execute('''create table EST_this_year(key INTEGER,
            symbol INTEGER, date INTEGER, period INTEGER,
            publish INTEGER, product INTEGER,unit INTEGER,
            type INTEGER, production INTEGER, Sales INTEGER)''')

conn.commit()
cur.close()


In [ ]:
conn = sqlite3.connect("codal.sqlite")
cur = conn.cursor()

# 1) create your tables (your existing CREATE TABLE code)
# ...

# 2) insert data from all_data
cols = [
    "key", "symbol", "date", "period", "publish",
    "product", "unit", "type", "production", "Sales"
]

for table_name, df in all_data.items():
    if df.height == 0:
        continue

    df_insert = df.select(cols)
    rows = df_insert.rows()
    placeholders = ",".join("?" for _ in cols)

    cur.executemany(
        f"INSERT INTO {table_name} ({', '.join(cols)}) VALUES ({placeholders})",
        rows
    )

conn.commit()
cur.close()
conn.close()


NameError: name 'all_data' is not defined

In [ ]:
all_data['report_last_year'].to_pandas().to_sql('report_last_year', conn, if_exists="replace", index=False, method=None)

7

In [ ]:
all_data.write_database(
    connection="sqlite://my.db",
    table_name="report_last_year",
    if_table_exists="replace"
)

AttributeError: 'dict' object has no attribute 'write_database'